# **Efficient Deep CNN-BiLSTM Model for Network Intrusion Detection**

**Implemented By:** Sarvesh Sridhar 19BAI1057

**Dataset used:** KDD CUP 1999

**Method used:** Deep CNN-BiDirectional LSTM Sequential Model





In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from itertools import cycle

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN,Bidirectional, BatchNormalization,Convolution1D,MaxPooling1D, Reshape, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

In [ ]:
#Loading training set into dataframe
train = pd.read_csv('C:/Users/KSGS/AI_CS_project/KDD/KDDTrain+.txt', header=None)
#Loading testing set into dataframe
test = pd.read_csv('C:/Users/KSGS/AI_CS_project/KDD//KDDTest+.txt', header=None)

In [ ]:
#Reset column names for training set
col = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
'num_access_files', 'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count', 'srv_count', 'serror_rate',
'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
'dst_host_srv_count', 'dst_host_same_srv_rate','dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'subclass', 'difficulty_level']

train.columns = col
test.columns = col
train.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,subclass,difficulty_level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [ ]:
train = train.drop('difficulty_level', 1) 
train.shape

<ipython-input-4-7bc21aa51425>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  train = train.drop('difficulty_level', 1)


(125973, 42)

In [ ]:
test = test.drop('difficulty_level', 1)
test.shape

<ipython-input-5-6f30eb685c60>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  test = test.drop('difficulty_level', 1)


(22544, 42)

In [ ]:
train.isnull().values.any()

False

In [ ]:
test.isnull().values.any()

False

In [ ]:
#One-hot encoding
def one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(each, 1)
    return df

cat_cols = ['protocol_type','service','flag']


#Merging train and test data
combined_data = pd.concat([train,test])

#Dropping subclass column
attack = combined_data['subclass']
combined_data.drop('subclass', axis=1, inplace = True)

#Applying one hot encoding to combined data
combined_data = one_hot(combined_data,cat_cols)

<ipython-input-8-d07736bc19e9>:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop(each, 1)


In [ ]:
#Function to min-max normalize
def normalize(df, cols):
    result = df.copy() # do not touch the original df
    for feature_name in cols:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        if max_value > min_value:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

#Normalizing training set
new_train_df = normalize(combined_data,combined_data.columns)
new_train_df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,3.558064e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
#Fixing labels for training set
classlist = []
check1 = ("apache2","back","land","neptune","mailbomb","pod","processtable","smurf","teardrop","udpstorm","worm")
check2 = ("ipsweep","mscan","nmap","portsweep","saint","satan")
check3 = ("buffer_overflow","loadmodule","perl","ps","rootkit","sqlattack","xterm")
check4 = ("ftp_write","guess_passwd","httptunnel","imap","multihop","named","phf","sendmail","Snmpgetattack","spy","snmpguess","warezclient","warezmaster","xlock","xsnoop")

DoSCount=0
ProbeCount=0
U2RCount=0
R2LCount=0
NormalCount=0

for item in attack:
    if item in check1:
        classlist.append("DoS")
        DoSCount=DoSCount+1
    elif item in check2:
        classlist.append("Probe")
        ProbeCount=ProbeCount+1
    elif item in check3:
        classlist.append("U2R")
        U2RCount=U2RCount+1
    elif item in check4:
        classlist.append("R2L")
        R2LCount=R2LCount+1
    else:
        classlist.append("Normal")
        NormalCount=NormalCount+1
        
#Appending class column to training set
new_train_df["Class"] = classlist
new_train_df.head()

<ipython-input-10-0f09b90e480b>:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  new_train_df["Class"] = classlist


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,Class
0,0.0,3.558064e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Normal
1,0.0,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Normal
2,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,DoS
3,0.0,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Normal
4,0.0,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Normal


In [ ]:
new_train_df["Class"].value_counts()

Normal    77232
DoS       53387
Probe     14077
R2L        3702
U2R         119
Name: Class, dtype: int64

In [ ]:
new_train_df.isnull().values.any()

False

In [ ]:
x = new_train_df.drop('Class', axis = 1)
y = new_train_df["Class"]

print("Shape of x: ", x.shape)
print("Shape of y: ", y.shape)

Shape of x:  (148517, 122)
Shape of y:  (148517,)


In [ ]:
oos_pred = []

kfold = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
kfold.get_n_splits(x,y)

10

In [ ]:
# combined_data = pd.read_csv('x_kdd.csv')
# y_train = pd.read_csv('y_kdd.csv')

# combined_data = combined_data[combined_data.columns[1:]]
# y_train = y_train['Class']

# print(combined_data.shape, y_train.shape)

In [ ]:
import tensorflow as tf

model = Sequential()
model.add(Convolution1D(64, kernel_size = 64, padding = 'same', activation = 'relu', input_shape = (122,1)))
model.add(MaxPooling1D(pool_size  =(10)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64,return_sequences = False)))
model.add(Reshape((128,1), input_shape = (128,)))
model.add(MaxPooling1D(pool_size = (5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(128,return_sequences = False)))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
for layer in model.layers:
    print(layer.output_shape)

(None, 122, 64)
(None, 12, 64)
(None, 12, 64)
(None, 128)
(None, 128, 1)
(None, 25, 1)
(None, 25, 1)
(None, 256)
(None, 256)
(None, 5)
(None, 5)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 122, 64)           4160      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 64)           0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 12, 64)           256       
 ormalization)                                                   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 reshape (Reshape)           (None, 128, 1)            0         
                                                        

In [ ]:
# combined_data_X.to_csv('x_kdd.csv')
# y_train.to_csv('y_kdd.csv')

In [ ]:
for train_index, test_index in kfold.split(x, y):
    # split
    train_X, test_X = x.iloc[train_index], x.iloc[test_index]
    train_y, test_y = y.iloc[train_index], y.iloc[test_index]
    
    x_columns_train = new_train_df.columns.drop('Class')
    x_train_array = train_X[x_columns_train].values
    x_train_1=np.reshape(x_train_array, (x_train_array.shape[0], x_train_array.shape[1], 1))
    
    dummies = pd.get_dummies(train_y)
    outcomes = dummies.columns
    num_classes = len(outcomes)
    y_train_1 = dummies.values
    
    x_columns_test = new_train_df.columns.drop('Class')
    x_test_array = test_X[x_columns_test].values
    x_test_2=np.reshape(x_test_array, (x_test_array.shape[0], x_test_array.shape[1], 1))
    
    dummies_test = pd.get_dummies(test_y) # Classification
    outcomes_test = dummies_test.columns
    num_classes = len(outcomes_test)
    y_test_2 = dummies_test.values
    
    with tf.device('/GPU:0'):
        model.fit(x_train_1, y_train_1,validation_data=(x_test_2,y_test_2), epochs=10, batch_size = 200)
    
    pred = model.predict(x_test_2)
    pred = np.argmax(pred,axis=1)
    y_eval = np.argmax(y_test_2,axis=1)
    score = metrics.accuracy_score(y_eval, pred)
    oos_pred.append(score)
    print("Validation score: {}".format(score))

Epoch 1/10
669/669 [==============================] - 27s 25ms/step - loss: 0.1243 - accuracy: 0.9620 - val_loss: 0.0732 - val_accuracy: 0.9757
Epoch 2/10
669/669 [==============================] - 15s 23ms/step - loss: 0.0596 - accuracy: 0.9804 - val_loss: 0.0648 - val_accuracy: 0.9807
Epoch 3/10
669/669 [==============================] - 15s 22ms/step - loss: 0.0490 - accuracy: 0.9835 - val_loss: 0.0507 - val_accuracy: 0.9827
Epoch 4/10
669/669 [==============================] - 15s 23ms/step - loss: 0.0412 - accuracy: 0.9861 - val_loss: 0.0372 - val_accuracy: 0.9870
Epoch 5/10
669/669 [==============================] - 15s 22ms/step - loss: 0.0348 - accuracy: 0.9880 - val_loss: 0.0365 - val_accuracy: 0.9881
Epoch 6/10
669/669 [==============================] - 16s 24ms/step - loss: 0.0327 - accuracy: 0.9885 - val_loss: 0.0337 - val_accuracy: 0.9884
Epoch 7/10
669/669 [==============================] - 15s 23ms/step - loss: 0.0301 - accuracy: 0.9893 - val_loss: 0.0383 - val_accuracy:

In [ ]:
oos_pred

[0.9886883921357393,
 0.990506329113924,
 0.9929302450848371,
 0.9934688930783733,
 0.9943441960678696,
 0.9933342310799892,
 0.9952194990573661,
 0.9942091441653761,
 0.9940744730994546,
 0.9930644401050435]

In [ ]:
cm = confusion_matrix(y_eval, pred, labels=[0,1,2,3,4])

In [ ]:
def count_unique(keys):
    uniq_keys = np.unique(keys)
    bins = uniq_keys.searchsorted(keys)
    return [uniq_keys, np.bincount(bins)]

y_pred = np.array(pred)
labels = print(dummies_test.columns)
print(count_unique(y_pred)[0])
count_unique(y_pred)[1]

Index(['DoS', 'Normal', 'Probe', 'R2L', 'U2R'], dtype='object')
[0 1 2 3 4]


array([5327, 7716, 1400,  399,    9], dtype=int64)

In [ ]:
print(cm)

[[5325   13    0    0    0]
 [   2 7670    4   47    0]
 [   0   10 1396    1    0]
 [   0   21    0  349    1]
 [   0    2    0    2    8]]


In [ ]:
from sklearn.metrics import classification_report
print('\nClassification Report\n')
print(classification_report(y_eval, pred, target_names = labels))


Classification Report

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5338
           1       0.99      0.99      0.99      7723
           2       1.00      0.99      0.99      1407
           3       0.87      0.94      0.91       371
           4       0.89      0.67      0.76        12

    accuracy                           0.99     14851
   macro avg       0.95      0.92      0.93     14851
weighted avg       0.99      0.99      0.99     14851



In [ ]:
dummies = pd.get_dummies(y)
outcomes = dummies.columns
num_classes = len(outcomes)
y_final = dummies.values
    
y_pred = model.predict(x.values)
pred = np.argmax(y_pred ,axis=1)
y_eval = np.argmax(y_final,axis=1)
score = metrics.accuracy_score(y_eval, pred)

print("Accuracy :", score)

Accuracy : 0.9953204010315317
